In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [16]:
def standardize_data_single_feature(data):
    scaler = StandardScaler()
#     meantemp_data_transformed = data['meantemp'].values.reshape(-1, 1)
#     meantemp_data_transformed_z_scaled = scaler.fit_transform(meantemp_data_transformed)
    print (data.columns)
    meantemp_data_transformed_z_scaled = scaler.fit_transform(data[['meantemp','humidity','wind_speed','meanpressure']])
    data[['meantemp','humidity','wind_speed','meanpressure']] = meantemp_data_transformed_z_scaled
    return data
climate_train_dataset = pd.read_csv("/kaggle/input/daily-climate-time-series-data/DailyDelhiClimateTrain.csv")
climate_test_dataset = pd.read_csv("/kaggle/input/daily-climate-time-series-data/DailyDelhiClimateTest.csv")
climate_train_dataset = standardize_data(climate_train_dataset)
climate_test_dataset = standardize_data(climate_test_dataset)
print (len(climate_train_dataset))
len(climate_test_dataset)

Index(['date', 'meantemp', 'humidity', 'wind_speed', 'meanpressure'], dtype='object')
Index(['date', 'meantemp', 'humidity', 'wind_speed', 'meanpressure'], dtype='object')
1462


114

##univariate forecasting using lstm

In [ ]:
def standardize_data_single_feature(data):
    scaler = StandardScaler()
    print (data.columns)
    meantemp_data_transformed_z_scaled = scaler.fit_transform(data[['meantemp']])
    data[['meantemp']] = meantemp_data_transformed_z_scaled
    return data,scaler_single
climate_train_dataset = pd.read_csv("/kaggle/input/daily-climate-time-series-data/DailyDelhiClimateTrain.csv")
climate_test_dataset = pd.read_csv("/kaggle/input/daily-climate-time-series-data/DailyDelhiClimateTest.csv")
climate_train_datase,scaler_single = standardize_data_single_feature(climate_train_dataset)
climate_test_dataset,scaler_single = standardize_data_single_feature(climate_test_dataset)
print (len(climate_train_dataset))
len(climate_test_dataset)

In [ ]:
def create_sequences(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)
train_meantemp_data = climate_train_dataset['meantemp'].values
test_meantemp_data = climate_test_dataset['meantemp'].values
time_steps = 10
X_train, y_train = create_sequences(train_meantemp_data, time_steps)
X_test, y_test = create_sequences(test_meantemp_data, time_steps)

def predict_using_lstm(X_train,y_train,no_of_features,time_steps,epochs,batch_size,scaler):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True,input_shape=(time_steps, no_of_features)))
    model.add(LSTM(units=50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=epochs,batch_size=batch_size)
    predicted_test_values = model.predict(X_test)
    predicted_test_values_original = scaler.inverse_transform(predicted_test_values)
    y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1))
    return predicted_test_values_original,y_test_original
predicted_test_values_original,y_test_original = predict_using_lstm()
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.plot(y_test_original, label='Real Test Values')
plt.plot(predicted_test_values_original, label='Predicted Test Values')
plt.title('LSTM Forecast vs Real Test Values')
plt.xlabel('Time')
plt.ylabel('Temperature (Original Scale)')
plt.legend()
plt.show()

##multivariate forecasting using lstm

In [ ]:
def standardize_data_multi_feature(data):
    scaler = StandardScaler()
    print (data.columns)
    meantemp_data_transformed_z_scaled = scaler.fit_transform(data[['meantemp','humidity','wind_speed','meanpressure']])
    data[['meantemp','humidity','wind_speed','meanpressure']] = meantemp_data_transformed_z_scaled
    return data, scaler
def create_multivariate_sequences(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps, :-1])  # All features except the target
        y.append(data[i + time_steps, 0])  # Target variable (e.g., meantemp)
    return np.array(X), np.array(y)
climate_train_dataset = pd.read_csv("/kaggle/input/daily-climate-time-series-data/DailyDelhiClimateTrain.csv")
climate_test_dataset = pd.read_csv("/kaggle/input/daily-climate-time-series-data/DailyDelhiClimateTest.csv")
climate_train_dataset,scaler_multi = standardize_data_multi_feature(climate_train_dataset)
climate_test_dataset,scaler_multi = standardize_data_multi_feature(climate_test_dataset)

train_meantemp_data = climate_train_dataset[['meantemp', 'humidity', 'wind_speed', 'meanpressure']].values
test_meantemp_data = climate_test_dataset[['meantemp', 'humidity', 'wind_speed', 'meanpressure']].values
time_steps = 10
X_train, y_train = create_multivariate_sequences(train_meantemp_data, time_steps)
X_test, y_test = create_multivariate_sequences(test_meantemp_data, time_steps)
epochs = 20
batch_size=32
predicted_test_values_original,y_test_original = predict_using_lstm(X_train,y_train,X_train.shape[2],time_steps,batch_size,scaler_multi)
plt.figure(figsize=(12, 6))
plt.plot(y_test_original, label='Real Test Values')
plt.plot(predicted_test_values_original, label='Predicted Test Values')
plt.title('LSTM Forecast vs Real Test Values')
plt.xlabel('Time')
plt.ylabel('Temperature (Original Scale)')
plt.legend()
plt.show()

In [ ]:
#shaping into (samples, timesteps,features)
X_train = X_train.reshape((X_train.shape[0], 10, 1))
X_test = X_test.reshape((X_test.shape[0], 10, 1))


In [ ]:
#multivariate forecasting using lstim

In [ ]:






time_steps = 10  # You can adjust this number based on how many past observations you want to use


X_train, y_train = create_multivariate_sequences(train_meantemp_data, time_steps)
X_test, y_test = create_multivariate_sequences(test_meantemp_data, time_steps)
print ("shape", X_train.shape)
print ("X.shape[1]", X_train.shape[1])
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2]))

model = Sequential()
model.add(LSTM(units=50, return_sequences=True,input_shape=(time_steps, X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(1))

# Step 5: Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Step 6: Train the model
model.fit(X_train, y_train, epochs=20,batch_size=32)

# # Step 7: Making predictions
# # You can predict future values by providing the model with the most recent sequence of time steps
# last_sequence = meantemp_data[-time_steps:].reshape(1, time_steps, 1)
# predicted_value = model.predict(last_sequence)

# # Inverse transform to get the predicted value back in original scale
# predicted_value_original = scaler.inverse_transform(predicted_value)
# print("Predicted next value:", predicted_value_original)

predicted_test_values = model.predict(X_test)
predicted_test_values_original = scaler.inverse_transform(predicted_test_values)
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1))
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.plot(y_test_original, label='Real Test Values')
plt.plot(predicted_test_values_original, label='Predicted Test Values')
plt.title('LSTM Forecast vs Real Test Values')
plt.xlabel('Time')
plt.ylabel('Temperature (Original Scale)')
plt.legend()
plt.show()

In [ ]:
def create_multivariate_sequences(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps, :-1])  # All features except the target
        y.append(data[i + time_steps, 0])  # Target variable (e.g., meantemp)
    return np.array(X), np.array(y)

In [ ]:
X.shape

In [ ]:

# !pip install -U numpy
import numpy
numpy.__version__

In [ ]:
    !pip install tensorflow